In [ ]:
using Dates, CSV, JuliaDB, Plots, PlotThemes, Interact, HTTP, SingularSpectrumAnalysis
using Revise, OnlineStats
theme(:dark)

# OnlineStats and Big Data Viz

## `Partition` and `IndexedPartition`

- Plot any-sized dataset.
- Here are two plots with **100 Million observations**:

In [ ]:
o = Partition(Series(Mean(), Extrema()))

y = randn()

for _ in 1:10^8
    fit!(o,  y += randn())
end

plot(o)

In [ ]:
o = IndexedPartition(Float64, KHist(10))

for _ in 1:10^8
    fit!(o,  (randn(), randn()))
end

plot(o)

# Kaggle's [Huge Stock Market Dataset](https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs)

- OHLC data for each stock/ETF (each is a separate CSV) traded in the US
- Just over 700MB

In [ ]:
path = "/Users/joshday/datasets/price-volume-data-for-all-us-stocks-etfs/Stocks/"

readdir(path)

In [ ]:
;head "$path/aapl.us.txt"

## Statistics/Plots Directly From CSV

In [ ]:
function plot_high_low(stock; kw...)
    o = IndexedPartition(Date, Extrema(), 500)
    for row in CSV.File("$path/$stock.us.txt") 
        fit!(o, [row.Date => row.Low, row.Date => row.High])
    end
    t = uppercase(stock) * " (nobs = $(nobs(o)))"
    plot(o; xlab="Date", title=t, legend=false, kw...)
end

In [ ]:
plot_high_low("aapl")

## ...But this loads the entire CSV!

Even though we are creating the plots with OnlineStats, the entire file is loaded into memory.

### `CSV.Rows`

- `CSV.Rows` lets you lazily read from a CSV file
    - **Minimal memory footprint**
    - At the cost of **no type inference** (everything is treated as `String`)

In [ ]:
function plot_high_low2(stock; kw...)
    o = IndexedPartition(Date, Extrema(), 500)
    for row in CSV.Rows("$path/$stock.us.txt")
        dt = Date(row.Date, "yyyy-mm-dd")
        low = parse(Float64, row.Low)
        hi = parse(Float64, row.High)
        fit!(o, [dt => low, dt => hi])
    end
    t = uppercase(stock) * " (nobs = $(nobs(o)))"
    plot(o; xlab="Date", title=t, legend=false, kw...)
end

In [ ]:
plot_high_low2("aapl")

In [ ]:
plot(plot_high_low2("aapl"), plot_high_low2("msft"), layout=(2,1), link=:x)

## Loading Multiple Datasets at Once

- Working with only one CSV at a time limits what kinds of analyses we can do.

In [ ]:
t = loadtable(path, filenamecol = :stock)

### Passing an `OnlineStat` as a reducer

In [ ]:
reduce(Mean(), t, select=:Open)

In [ ]:
temp = groupreduce(Mean(), t, :stock, select=:Open)
select(temp, (1, 2 => value))

### Make All Plots at Once

In [ ]:
allplots = groupreduce(IndexedPartition(Date, Extrema()), t, :stock, select=(:Date, :Close))

In [ ]:
@manipulate for s in ["aapl", "msft", "ibm", "googl", "nflx"]
    t2 = filter(==("$s.us.txt"), allplots, select=:stock)
    plot(t2[1].IndexedPartition, title=s)
end

## Singular Spectrum Analysis

- Extract seasonal components out of a time series

In [ ]:
temp = filter(==("aapl.us.txt"), t, select=:stock)

yt, ys = analyze(select(temp, :Close), 90)
plot(yt, lab="Trend")
plot!(ys, lab="Season")

## Autocorrelation

In [ ]:
temp = filter(==("aapl.us.txt"), t, select=:stock)


plot(fit!(AutoCov(365), select(temp, :Close)))

# TrueFX API

In [ ]:
function get_data(q = "")
    endpoint = "https://webrates.truefx.com/rates/connect.html?f=csv&$q"
    hdr = [:pair, :utc, :big_bid_figure, :bid_points, :offer_bid_figure,
           :offer_points, :high, :low, :open]
    r = HTTP.get(endpoint)
    CSV.read(r.body; header=hdr, footerskip=1)
end

In [ ]:
get_data("c=EUR/USD")

In [ ]:
o = IndexedPartition(Int, Extrema(), 10)

while true
    IJulia.clear_output(true)
    df = get_data("c=USD/JPY")
    fit!(o, (df[1, :utc], df[1, :bid_points]))
    display(plot(o, xlab="UTC Time", ylab="Bid Points", title="nobs = $(nobs(o))"))
end